# Self-RAG

Source : https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/

In [14]:
# !pip install -q langchain openai pandas PyPDF2 chromadb tiktoken langchain-openai langchain-community pypdf rank_bm25
# !pip install -q langgraph langsmith

In [15]:
import os

import dotenv
dotenv.load_dotenv("/home/chougar/Documents/GitHub/Formation_datascientest/DL-NLP/.env")

True

## Loas the data

In [16]:
# libraries and models setup
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.retrievers import SelfQueryRetriever
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_ollama.llms import OllamaLLM
from time import time as timing


#### LLM & modele d'embedding à utiliser


In [17]:
# model_qa_name="qwen2.5:3b"
# model_qa_name="mistral:latest"
# model_qa_name="deepseek-r1:8b"
#model_qa_name="hf.co/bartowski/Falcon3-7B-Instruct-GGUF:Q4_0"
# model_qa_alias="falcon3-7b-mamba"
# llm_qa = OllamaLLM(model=model_qa_name, temperature=0.2)

model_qa_name="gpt-4o-mini"
model_qa_alias=model_qa_name
llm_qa = ChatOpenAI(model_name=model_qa_name, temperature=0.2)



# Embeddings model definition
# model_emb_name='text-embedding-ada-002'
model_emb_name="text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=model_emb_name)


In [18]:
pdf_file_path = './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf'

# Create a loader instance
loader = PyPDFLoader(pdf_file_path)

# Load the data from the PDF
data = loader.load()

#### Step 3: Split  PDF en fragments
Basé sur l'expérience du notebook `rag-v2-gridSearch`, le meilleur modèle et chunk size/overlap sont `text-embedding-ada-002` et 2000 / 400


In [19]:
x=2000
text_splitter = RecursiveCharacterTextSplitter(chunk_size=x, chunk_overlap=x*0.2)

split_docs = text_splitter.split_documents(data)

In [20]:
# nettoyage
bruits=["Planète Urgence | FOREST Programme"]
for doc in split_docs:
    for bruit in bruits:
        if bruit in doc.page_content:
            doc.page_content=doc.page_content.replace(bruit, "")
# del empty docs
split_docs = [doc for doc in split_docs if len(doc.page_content)>0]

#### Step 4: Creation d'une BDD Faiss

In [21]:

faiss_db = FAISS.from_documents(split_docs, embedding_model,)


retriever_faiss = faiss_db.as_retriever(
    search_kwargs={
        "k": 10, 
        "fetch_k": 30
    },
    search_type= "mmr"
)


# Test
query = "Context, environment, project rationale and challenges"

t=timing()
result = retriever_faiss.invoke(query)
print(f"t2 :{timing()-t}")
print(f"faiss:")
display(result)

t2 :0.5328977108001709
faiss:


[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 17}, page_content='\nAnother issue that has also had a major impact on life in Indonesia, including\nEast Kalimantan, is the Covid 19 global pandemic that has attacked the\nentire world since early 2020.  The  Covid 19 pandemic has had a major\nimpact on life in Indonesia. East Kalimantan is a province outside Java Island\nwith  the  highest  rate  of  exposure  to  Covid,  which  has  resulted  in  the\ngovernment imposing a lockdown and restrictions on community activities.\nAt the beginning of 2022, community activities began to return to normal,\nbut a new variant emerged, namely Omicron, which spread very quickly.\nFacing a pandemic situation that has not ended, of course, the project must\ncontinue  but  still  pay  attention  to  security,  safety,  and  practice  health\nprotocols.\n3. Strategy & theory of change\nThe  three  years  project  aims  to  contribute  to  restore  the  degr

In [22]:
from langchain.retrievers import BM25Retriever, TFIDFRetriever
from sentence_transformers import CrossEncoder

sparse_retriever_tfidf = TFIDFRetriever.from_documents(
    documents=split_docs,
    k=10,
    tfidf_params={"min_df":1,
            "max_df":2,
            "ngram_range":(1, 2)}
    )


# question="Context, environment, project rationale and challenges"
# question='What are the key contextual factors, environmental conditions, project rationale, and potential challenges that may impact the success of the project, including aspects related to biodiversity, climate change impacts, socio-economic context, and health considerations?'
question="Number of People Concerned"
t=timing()
results=sparse_retriever_tfidf.invoke(question)
print(f"time tfidf:", timing()-t)
display(results)



time tfidf: 0.0031669139862060547


[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 40}, page_content="\nlocal  communities  and  the  public  stakeholders  concerned  (villages  and\ndistricts) in order to replicate these activities with the other villages of the\nMahakam Delta.\nThe tree protection from palm tree leaves and net that successfully works to\nprotect tree in previous project will be documented and spread to the key\nrestoration actors for possible replication in their project. In addition, PU\nIndonesia will manage and distribute the publication and knowledge from the\nproject  to  other  parties.  Module  and  material  environmental  education\ncampaign will be used in the scale up project or other locations.\nTable 3. Summary of our lessons learned, and best practices developed on\nprevious projects in the Mahakam Delta\n<table>\nActivities Best practices\n(+)\nLessons learned\n(-)\nChange maker in\nthis new project \nRaising awareness\nin schools \n1/Raisin

Lexical retriever (basé sur le key word matching)

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


import string

def remove_punctuation(text):
    text_no_punctuation = text.translate(str.maketrans('', '', string.punctuation))
    words = text_no_punctuation.split()
    return words

def keyword_association_retriever(query, top_k=10, documents=split_docs, print_keywords=False):
    query_keywords = llm_extract_keyworks.invoke({"question":query})
    # print(query_keywords)
    query_keywords = remove_punctuation(query_keywords)  # Liste des mots-clés dans la requête
    if print_keywords:
        print(query_keywords)
    results = []


    for doc in documents:
        doc_text = doc.page_content.lower()
        # Compter les cooccurrences des mots-clés dans le document
        keyword_matches = sum(1 for word in query_keywords if word.lower() in doc_text)
        # Pondérer le score par la densité des mots-clés
        try:
            score = keyword_matches / len(doc_text.split())
        except Exception as e:
            print(e)
            score=0

        results.append((doc, score))

    # Trier les résultats par score
    ranked_results = sorted(results, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked_results[:top_k]]

system_prompt_exact="""Extract the most relevant keywords from the following text to facilitate searching for important documents containing these terms. 
    Only provide the extracted keywords, without additional text or explanations.
"""

system_prompt_synonyms="""Extract the most relevant keywords from the following text to facilitate searching for important documents.  

- For each extracted keyword, also include its closest synonyms.  
- Return all keywords and synonyms in a **single flat list**, without categorization, explanations, or additional text.  
- Ensure the extracted terms are meaningful and relevant to the text's context.  

### **Output Format:**  
A **comma-separated** flat list of keywords and their closest synonyms.  
"""

prompt_extract_keyworks = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_exact),
        ("human", "{question}"),
    ]
)
llm_extract_keyworks = prompt_extract_keyworks | llm_qa | StrOutputParser()
resp=llm_extract_keyworks.invoke(question)
remove_punctuation(resp)

print(f"Question: {question}")
keyword_association_retriever(query=question, top_k=10, print_keywords=True)

Question: Number of People Concerned
['Number', 'People', 'Concerned']


[Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 18}, page_content='\nFeature 3. Theory of Change\n4. Beneficiaries\nNumber of direct beneficiaries of the pilot project: 3245 people with the\nproportion of 30% women and 70% men. \nNumber of indirect beneficiaries: 3000 people by assuming at least the\nproject will give benefit indirectly to 1500 people per location including in\nEast Kalimantan and Indonesia.\nThe target groups include: \n- School children (primary schools and secondary schools)\n- Teachers (primary school teachers)'),
 Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 38}, page_content='\ndecide to contribute on the project. By doing this, the project aims to place \nthe local people not as object the project, but the subject of project since \nthey also contribute to design the activities of the project. \nMoreover, regarding the new capital, the project expects to bring the voice

## Document relevent assessement

In [24]:
### Retrieval Grader


from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "What is the specific country and city where the project is being implemented?"
docs = retriever_faiss.invoke(question)
doc_txt = docs[3].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
print(doc_txt)

binary_score='yes'

Feature 3. Theory of Change
4. Beneficiaries
Number of direct beneficiaries of the pilot project: 3245 people with the
proportion of 30% women and 70% men. 
Number of indirect beneficiaries: 3000 people by assuming at least the
project will give benefit indirectly to 1500 people per location including in
East Kalimantan and Indonesia.
The target groups include: 
- School children (primary schools and secondary schools)
- Teachers (primary school teachers)


#### Hybrid retrieval

In [25]:
from langchain_core.runnables import chain


dense_retriever = retriever_faiss
sparse_retriever = sparse_retriever_tfidf

from transformers import pipeline

pipe_reranker = pipeline("question-answering", model="deepset/roberta-base-squad2", device=0)

@chain
def hybrid_retrieval(query, top_k=8, dense_retriever=dense_retriever, sparse_retriever=sparse_retriever):
    # query = query["question"]
    dense_results = dense_retriever.get_relevant_documents(query)
    sparse_results = sparse_retriever.get_relevant_documents(query)

    # Fusionner
    combined_results = dense_results + sparse_results + keyword_association_retriever(query)
    

    # Supprimer les doublons
    unique_results = {doc.page_content: doc for doc in combined_results}.values()

    
    # Reranking with crossencoder
    # reranked_results = reranker.predict([(query, doc.page_content) for doc in unique_results])
    # sorted_results = sorted(zip(unique_results, reranked_results), key=lambda x: x[1], reverse=True)
    print(f"nb resultats uniques avant reranking: {len(unique_results)}")
    # Reranking with Q/A model
    reranked_results=[{"score": pipe_reranker(query, doc.page_content)["score"], "doc": doc} for doc in unique_results]
    

    # Filtrer sur 1ere moitiée
    n=min(len(reranked_results), top_k)
    sorted_results=sorted(reranked_results, key=lambda x: x["score"]>0.25, reverse=True)[:n]
    # ajout du score pour analyse
    res=[]
    for doc in sorted_results:
        doc["doc"].metadata["score"]=doc["score"]
        res.append(doc["doc"])
    sorted_results=res    
    
    print(f"nb resultats uniques après reranking: {len(sorted_results)}")
    


    # return [doc for doc, _ in sorted_results[:top_k]]
    return sorted_results

Device set to use cuda:0


In [29]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the retrieval step
retrieval_step = RunnablePassthrough.assign(
    context=lambda x: hybrid_retrieval.invoke(x["question"])
)

# Define the prompt template
rag_prompt = ChatPromptTemplate.from_template("""
    Answer the question based **only** on the provided context.  

    - If the context contains enough information to provide a complete or partial answer, use it to formulate a detailed and factual response.  
    - If the context lacks relevant information, respond with: "I don't know."  

    ### **Context:**  
    {sources}  

    ### **Question:**  
    {question}  

    ### **Answer:**  
    Provide a clear, factual, and well-structured response based on the available context. Avoid speculation or adding external knowledge.  
""")

# Define the LLM and output parser

output_parser = StrOutputParser()

# Define the final pipeline
rag_chain_with_sources = (
    RunnableParallel({
        "question": RunnablePassthrough(),  # Pass the question through
        "sources": retrieval_step,         # Retrieve documents
    })
    | {
        "answer": rag_prompt | llm_qa | output_parser,  # Generate the answer
        "sources": lambda x: x["sources"]           # Pass the retrieved documents
    }
)

# Example question
question = "What is the specific country and city where the project is being implemented?"

# Invoke the pipeline
generation = rag_chain_with_sources.invoke({"question": question})

# Output the results
print("Answer:", generation["answer"])
print("Sources:")
generation["sources"]

nb resultats uniques avant reranking: 25


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


nb resultats uniques après reranking: 8
Answer: The project is being implemented in Indonesia, specifically in the city of Samarinda and the surrounding areas of Paser District (Adang Bay village) and Kutai Kartanegara district (Delta Mahakam) in East Kalimantan Province.
Sources:


{'question': 'What is the specific country and city where the project is being implemented?',
 'context': [Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 0, 'score': 0.9597622752189636}, page_content='MANGROVE RESTORATION AND\nLOCAL DEVELOPMENT PROJECT\nFOREST PROGRAMME\nProject title: MAHAKAM\nCountry: Indonesia\nImplementing  partners:  Yayasan\nMangrove Lestari & Pokja Pesisir'),
  Document(metadata={'source': './data/PROJECT DOCUMENT MAHAKAM 2023-2025_balise.pdf', 'page': 19, 'score': 0.3036922812461853}, page_content='\n- Community members (villagers, consists of fish farmers, women group,\nand youth)\n- Village officials\n- Stakeholders  from  various  institutions  (government  institutions,\nuniversities, and non-government organizations)\n- Public audience in general (reached by Media) \n Other potential groups: \n- High school and university students\n- Environmental activists\n5. Location\nThe location of the project is in Paser 

## Hallucination answer

In [31]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""

hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

## Accurate answer

In [32]:
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

## Reformulation of the initial query

In [33]:
system="""
    You are a question rewriter tasked with improving input questions to optimize them for vector store retrieval. 
    Your mission is to refine, rephrase, and enhance the provided questions to ensure they are:
    * Clear and easy to understand.
    * Focused.
    * Optimized for effective retrieval by removing ambiguities.
    * Written in an interrogative form while preserving the original intent.

    #### Input Fields to Rework:
    Example 1:
    * Project Description: Reword questions that focus on the project’s overall scope and objectives.
    Example 2:
    * Country and City: Refine questions to specifically inquire about the project’s location.
    Example 3:
    * Target Beneficiaries: Enhance questions to clarify the population or group that benefits from the project.    
    * Number of People Concerned: Rework questions to quantify how many people the project impacts.
    Example 4:
    * Context, Environment, Project Rationale, Challenges, include these topics to enhance the final output:
            * Environmental Context
            * Biodiversity
            * Climate Change impacts
            * Socio-Economic Context
            * Economic, Demographic, Community impacts
            * Institutional context
            * Health context
        Rephrase questions that ask for background information, challenges, and the reasoning behind the project.

    Example 5:
    * Project Start Date / End Date: Rework questions regarding the project’s timeline.
    Example 6:
    * Financial Information:
        * Project Budget: Reword questions about the overall project budget.
        * Total Project Cost: Rephrase inquiries about the total cost of the project.
        * Donation Request Amount: Refine questions asking about the amount of funding requested.
        * Provisional Project Budget: Rework questions about the detailed provisional budget for the project.
        * Current Year Budget: Enhance questions related to the budget specific to the current year.


    #### Response Format:
    For each input question, rephrase it in a clear, and interrogative form, optimized for vector store retrieval. Return only the reworked question.
"""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
print(f"Initial question:\n{question}")
question_rewriter.invoke({"question": question})

Initial question:
What is the specific country and city where the project is being implemented?


'Which country and city is the project being implemented in?'

## Functions for langGraph

In [37]:
from pprint3x import pprint
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever_faiss.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain_with_sources.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

## Build the graph

In [38]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

## Call the workflow

In [39]:
from pprint3x import pprint

# Run
# question="Number of people concerned"
question="Context, environment, project rationale and challenges"
# question="Project start date / end date"
# question="Country and city"
inputs = {"question": question}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        pprint(value.keys, indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
<built-in method keys of dict object at 0x7f1c35a65d40>
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
<built-in method keys of dict object at 0x7f1c358821c0>
'\n---\n'
---GENERATE---
nb resultats uniques avant reranking: 23


/home/chougar/miniconda3/lib/python3.11/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


nb resultats uniques après reranking: 8
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
<built-in method keys of dict object at 0x7f1c35907980>
'\n---\n'
{'answer': 'The project outlined in the context aims to restore the degraded '
           'mangrove ecosystem in the Mahakam Delta and Adang Bay areas of '
           'East Kalimantan, Indonesia. This initiative is crucial due to '
           'several pressing challenges:\n'
           '\n'
           '1. **Impact of Covid-19**: The Covid-19 pandemic has significantly '
           'affected life in Indonesia, particularly in East Kalimantan, which '
           'has the highest exposure rate. The pandemic led to '
           'government-imposed lockdowns and restrictions on community '
           'activities, complicating project implementation. Although '
           'activities began to normalize in early 

In [44]:
from graphviz import Digraph

# Create a Graphviz Digraph
dot = Digraph(comment="LangGraph Workflow")

# Add nodes
dot.node("START", "START")
dot.node("retrieve", "Retrieve")
dot.node("grade_documents", "Grade Documents")
dot.node("generate", "Generate")
dot.node("transform_query", "Transform Query")
dot.node("END", "END")

# Add edges
dot.edge("START", "retrieve")
dot.edge("retrieve", "grade_documents")
dot.edge("grade_documents", "transform_query", label="transform_query")
dot.edge("grade_documents", "generate", label="generate")
dot.edge("transform_query", "retrieve")
dot.edge("generate", "generate", label="not supported")
dot.edge("generate", "END", label="useful")
dot.edge("generate", "transform_query", label="not useful")

# Render the graph
dot.render("workflow_graph", format="png", cleanup=True)
print("Graph saved as workflow_graph.png")

Graph saved as workflow_graph.png
